In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform
%matplotlib inline
plt.style.use('ggplot')

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
data_test = pd.read_csv("../data/raw/credit_test.csv", encoding='cp1251', sep=';')

In [4]:
data_test.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count
0,170747,F,48,MAR,UMN,"16558,00",10,1.10,"0,370409",GRD,ХАКАСИЯ РЕСП,34000,2.0,0.0
1,170748,M,29,MAR,SPC,"7028,72",6,1.40,"0,377281",SCH,ОБЛ КЕМЕРОВСКАЯ,19000,3.0,0.0
2,170749,F,20,UNM,SPC,"13019,00",10,1.60,"0,538515",SCH,ОБЛ БЕЛГОРОДСКАЯ,25000,1.0,0.0
3,170750,F,41,MAR,SPC,"9379,00",10,1.10,"0,482520",GRD,ОБЛ ИРКУТСКАЯ,30000,0.0,0.0
4,170751,F,31,MAR,SPC,"13990,00",6,1.43,"0,485914",GRD,ОБЛ КИРОВСКАЯ,20000,2.0,0.0


In [5]:
data_train = pd.read_csv("../data/raw/credit_train.csv", encoding='cp1251', sep=';')

In [6]:
data_train.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,"59998,00",10,1.6,"0,770249",GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,"10889,00",6,1.1,"0,248514",GRD,МОСКВА,43000.0,2.0,0.0,0
2,3,M,32,MAR,SPC,"10728,00",12,1.1,"0,459589",SCH,ОБЛ САРАТОВСКАЯ,23000.0,5.0,0.0,0
3,4,F,27,DIV,SPC,"12009,09",12,1.1,"0,362536",GRD,ОБЛ ВОЛГОГРАДСКАЯ,17000.0,2.0,0.0,0
4,5,M,45,MAR,SPC,"16908,89",10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,25000.0,1.0,0.0,0


In [7]:
def change(x):
    #print(type(x))
    s = x.split(',')
    return s[0] + '.' + s[1]
    

In [78]:
def split_data(data_transformed):
    data = data_transformed.drop(['open_account_flg', 'client_id'], axis = 1)
    data = pd.get_dummies(data, columns=['education', 'job_position', 'marital_status', 'gender', 'tariff_id', 'living_region'])
    target = data_transformed['open_account_flg']
    print(type(data))
    data = data.fillna(0)
    data[['score_shk', 'credit_sum']] = data[['score_shk', 'credit_sum']].apply(lambda x: map(change, x))
    data[['score_shk', 'credit_sum']] = data[['score_shk', 'credit_sum']].apply(pd.to_numeric)
    x_train, x_val, y_train, y_val = cross_validation.train_test_split(data, target, test_size=0.15, random_state=123, stratify = target)
    return x_train, x_val, y_train, y_val

In [79]:
def transform_data(data):
    data_transformed = data
    data_transformed['gender'] = students['sex'] == "M"
    return data_transformed

In [80]:
x_train, x_val, y_train, y_val = split_data(data_train)

<class 'pandas.core.frame.DataFrame'>


In [30]:
x_train.head()

,age,credit_sum,credit_month,score_shk,monthly_income,credit_count,overdue_credit_count,education_ACD,education_GRD,education_PGR,...,living_region_ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -,living_region_ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП,living_region_ЧУВАШСКАЯ РЕСП,living_region_ЧУВАШСКАЯ РЕСПУБЛИКА,living_region_ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ,living_region_ЧУКОТСКИЙ АО,living_region_ЭВЕНКИЙСКИЙ АО,living_region_ЯМАЛО-НЕНЕЦКИЙ АО,living_region_ЯРОСЛАВСКАЯ ОБЛ,living_region_ЯРОСЛАВСКАЯ ОБЛАСТЬ
169317,49,16279.0,10,0.465462,30000.0,3.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58535,42,53110.8,14,0.645696,32000.0,3.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116190,53,20257.0,24,0.447536,20000.0,2.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
167236,29,9592.0,6,0.211393,20000.0,2.0,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
42798,38,68989.0,10,0.442669,110000.0,2.0,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
rf_clf = RandomForestClassifier(class_weight = "balanced")
rf_param_dist ={"n_estimators" : np.arange(10, 100, 10),
                  "max_depth": sp_randint(1, 31),
                  "max_features": sp_randint(1, 10),
                  "min_samples_leaf": sp_randint(1, 11),
                  "max_features" : ["auto", "sqrt", "log2"],
                  "criterion": ["gini", "entropy"]}

n_iter_search = 250
rf_random_search = RandomizedSearchCV(rf_clf, param_distributions = rf_param_dist,
                                   n_iter = n_iter_search, random_state = 123, cv=5)
rf_random_search.fit(x_train, y_train)
rf_clf = rf_random_search.best_estimator_
rf_val_score = rf_clf.score(x_val, y_val)
print("Best randomized search score - %s" % rf_random_search.best_score_)
print("Validation score - %s" % rf_val_score)

Best randomized search score - 0.761376614249
Validation score - 0.761083455344


In [97]:
from sklearn.metrics import roc_auc_score
y_predict = rf_clf.predict(x_val)
print(roc_auc_score(y_val, y_predict))

0.666673807259


In [84]:
def prepare_data(data_transformed):
    data = data_transformed.drop(['client_id'], axis = 1)
    data = pd.get_dummies(data, columns=['education', 'job_position', 'marital_status', 'gender', 'tariff_id', 'living_region'])
    target = data_transformed['client_id']
    data = data.fillna(0)
    data[['score_shk', 'credit_sum']] = data[['score_shk', 'credit_sum']].apply(lambda x: map(change, x))
    data[['score_shk', 'credit_sum']] = data[['score_shk', 'credit_sum']].apply(pd.to_numeric)
    return data, target

In [87]:
def get_best_features(rf_clf, kol):
    t = []
    for i in range(0, len(x_train.columns)):
        t.append((x_train.columns[i], rf_clf.feature_importances_[i]))
    sorted_by_second = sorted(t, key=lambda tup: tup[1], reverse=True)
    best_features = [x[0] for x in sorted_by_second][:kol]
    return best_features

In [90]:
def get_result(rf_clf, best_features):
    from sklearn.metrics import roc_auc_score
    y_predict = rf_clf.predict(x_val[best_features])
    print(roc_auc_score(y_val, y_predict))

In [85]:
def get_ans_csv(model, best_features):
    x_test, clients_id = prepare_data(data_test)
    x_test_selected = x_test[best_features]
    y_test = model.predict(x_test_selected)
    ans = pd.DataFrame(
    {'_ID_': clients_id,
     '_VAL_': y_test,
    })
    ans.to_csv('./ans.csv', index=False)

In [74]:
{'criterion': 'gini',
 'max_depth': 29,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'n_estimators': 70}

{'criterion': 'gini',
 'max_depth': 29,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'n_estimators': 70}

In [75]:
rf_best = RandomForestClassifier(n_estimators=70, max_depth=29, min_samples_leaf=1, n_jobs=-1)

In [106]:
rf_best.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=29, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=70, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [113]:
best_features = get_best_features(rf_clf, 230)

check available columns

In [114]:
xx = x_test[best_features]

In [92]:
get_result(rf_best, x_train.columns)

0.522467954367


In [96]:
get_result(rf_clf, x_train.columns)

0.666673807259


In [115]:
rf_clf.fit(x_train[best_features], y_train)
get_result(rf_clf, best_features)

0.661621820664


In [100]:
get_ans_csv(rf_clf, best_features)

In [116]:
import cPickle

with open("./rf_model", 'wb') as f:
    cPickle.dump(rf_clf, f)